In [49]:
%pip install mxnet

     ---------------------------------------- 0.0/5.1 MB ? eta -:--:--
     ------------------ --------------------- 2.4/5.1 MB 12.3 MB/s eta 0:00:01
     -------------------------------------- - 5.0/5.1 MB 13.1 MB/s eta 0:00:01
     ---------------------------------------- 5.1/5.1 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/33.1 MB ? eta -:--:--
   -- ------------------------------------- 2.4/33.1 MB 11.2 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/33.1 MB 10.9 MB/s eta 0:00:03
   -------- ------------------------------- 7.1/33.1 MB 10.9 MB/s eta 0:00:03
   ----------- ---------------------------- 9.4/33.1 MB 11.1 MB/s eta 0:00:03
   -------------- ------------------------- 12.3/33.1 MB 11.3 MB/s eta 0:00:02
   ------------------ --------------------- 14.9/33.1 MB 11.5 MB/s eta 0:00:02
   --------------------- ------------------ 17.6/3

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Running from numpy source directory.
      C:\Users\TimPr\AppData\Local\Temp\pip-install-v4u5bg9b\numpy_175eb245dee94c0696ae7f12ea58f1a8\numpy\distutils\misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\TimPr\AppData\Local\Temp\pip-install-v4u5bg9b\numpy_175eb245dee94c0696ae7f12ea58f1a8\setup.py", line 419, in <module>
          setup_package()
        File "C:\Users\TimPr\AppData\Local\Temp\pip-install-v4u5bg9b\numpy_175eb245dee94c0696ae7f12ea58f1a8\setup.py", line 398, in setup_package
          from numpy.distutils.core import setup
        File "C:\Users\TimPr\AppData\Local\Temp\pip-install-v

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator
from gluonts.mx import Trainer

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

from sklearn.model_selection import (
    train_test_split,
    TimeSeriesSplit,
    KFold,
    StratifiedGroupKFold,
    GroupKFold,
    StratifiedKFold,
)

ModuleNotFoundError: No module named 'mxnet'

In [27]:
df = pd.read_csv('../Data/merged_cleaned.csv',
                 index_col= [0],
                 parse_dates= [0])



In [9]:
df.columns

Index(['Gesamt (Netzlast) [MWh] Berechnete Auflösungen',
       'Deutschland/Luxemburg [€/MWh] Originalauflösungen',
       'Biomasse [MWh] Berechnete Auflösungen',
       'Wasserkraft [MWh] Berechnete Auflösungen',
       'Wind Offshore [MWh] Berechnete Auflösungen',
       'Wind Onshore [MWh] Berechnete Auflösungen',
       'Photovoltaik [MWh] Berechnete Auflösungen',
       'Sonstige Erneuerbare [MWh] Berechnete Auflösungen',
       'Kernenergie [MWh] Berechnete Auflösungen',
       'Braunkohle [MWh] Berechnete Auflösungen',
       'Steinkohle [MWh] Berechnete Auflösungen',
       'Erdgas [MWh] Berechnete Auflösungen',
       'Pumpspeicher [MWh] Berechnete Auflösungen_y',
       'Sonstige Konventionelle [MWh] Berechnete Auflösungen', 'Holiday',
       'date', 'hour', 'dayofweek', 'day_name', 'quarter', 'month', 'year',
       'dayofyear', 'dayofmonth', 'date_offset', 'hour_sin', 'hour_cos',
       'dayofweek_sin', 'dayofweek_cos', 'month_sin', 'month_cos',
       'dayofyear_sin', 'd

In [29]:
df = df.iloc[:,0:14]

In [35]:
df.index[int(52621 * 0.8)], df.index[int(0 * 0.8)]

(Timestamp('2023-08-20 01:00:00'), Timestamp('2018-11-01 00:00:00'))

In [16]:
df_input = df.reset_index(drop=True).T.reset_index()

In [18]:
ts_code = df_input["index"].astype("category").cat.codes.values

In [22]:
df_train = df_input.iloc[:, 1:int(52621*0.8)]
df_test = df_input.iloc[:,int(52621*0.8):]

In [37]:
freq = "60min"
start_train = pd.Timestamp("2018-11-01 00:00:00") 
start_test = pd.Timestamp("2023-08-20 01:00:00") 

In [40]:
target_columns = [
    "Gesamt (Netzlast) [MWh] Berechnete Auflösungen",
    "Deutschland/Luxemburg [€/MWh] Originalauflösungen",
    "Biomasse [MWh] Berechnete Auflösungen",
    "Wasserkraft [MWh] Berechnete Auflösungen",
    "Wind Offshore [MWh] Berechnete Auflösungen",
    "Wind Onshore [MWh] Berechnete Auflösungen",
    "Photovoltaik [MWh] Berechnete Auflösungen",
    "Sonstige Erneuerbare [MWh] Berechnete Auflösungen",
    "Kernenergie [MWh] Berechnete Auflösungen",
    "Braunkohle [MWh] Berechnete Auflösungen",
    "Steinkohle [MWh] Berechnete Auflösungen",
    "Erdgas [MWh] Berechnete Auflösungen",
    "Pumpspeicher [MWh] Berechnete Auflösungen_y",
    "Sonstige Konventionelle [MWh] Berechnete Auflösungen"
]

In [42]:
# Extract the target data for training and testing
train_target = df_train.values.T  # Shape: (num_series, num_time_steps)
test_target = df_test.values.T

In [46]:
# Set the frequency as a string variable (useful when creating the dataset)
freq = "60min"

estimator = DeepAREstimator(freq=freq,
                            context_length= 672,
                            prediction_length= 672,
                            use_feat_static_cat = True,
                            cardinality= [1],
                            num_layers= 2,
                            num_cells = 32,
                            cell_type = "lstm",
                            trainer = Trainer(epochs=5))


NameError: name 'Trainer' is not defined